In [29]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import glob

import healpy

import numpy as np

from astropy.table import Table
from astropy.io import fits

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Filter DECaLS Sweep catalog through HSC FDFC healpix mask

- Using local DR9SV catalog as a test
- Filter through HSC S19A internal release

In [64]:
sweep_dir = '/Volumes/astro5/massive/decals/dr9sv/sweep/'

hsc_dir = '/Volumes/astro5/massive/s19a/'

s19a_fdfc_mask = os.path.join(
    hsc_dir, 'mask/fdfc/s19a_fdfc_hp_contarea_izy-gt-5_trimmed.fits')

### Find all the sweep catalog

In [8]:
sweep_list = glob.glob(sweep_dir + '/sweep*.fits')

In [49]:
sweep_test = Table.read(sweep_list[-1])

In [50]:
print(len(sweep_test))

36778


In [11]:
print(sweep_test.colnames)

['RELEASE', 'BRICKID', 'BRICKNAME', 'OBJID', 'TYPE', 'RA', 'DEC', 'RA_IVAR', 'DEC_IVAR', 'DCHISQ', 'EBV', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4', 'NOBS_G', 'NOBS_R', 'NOBS_Z', 'NOBS_W1', 'NOBS_W2', 'NOBS_W3', 'NOBS_W4', 'RCHISQ_G', 'RCHISQ_R', 'RCHISQ_Z', 'RCHISQ_W1', 'RCHISQ_W2', 'RCHISQ_W3', 'RCHISQ_W4', 'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z', 'FRACFLUX_W1', 'FRACFLUX_W2', 'FRACFLUX_W3', 'FRACFLUX_W4', 'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z', 'FRACIN_G', 'FRACIN_R', 'FRACIN_Z', 'ANYMASK_G', 'ANYMASK_R', 'ANYMASK_Z', 'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z', 'WISEMASK_W1', 'WISEMASK_W2', 'PSFSIZE_G', 'PSFSIZE_R', 'PSFSIZE_Z', 'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z', 'GALDEPTH_G', 'GALDEPTH

In [70]:
def read_healpix_mask(mask_file):
    """Read the FITS format healpix FDFC mask."""
    return healpy.read_map(mask_file, nest=True, dtype=np.bool)
    
def list_all_sweep_catalogs(sweep_dir, verbose=True):
    """Gather a list of pathes to all SWEEP catalogs."""
    sweep_list = glob.glob(sweep_dir + '/sweep*.fits')
    if verbose:
        print("# Find {:d} SWEEP files".format(len(sweep_list)))
    return sweep_list

def filter_hsc_fdfc_mask(cat, fdfc_mask, ra='RA', dec='DEC', verbose=True):
    """Filter a catalog through HSC FDFC mask."""
    # Read the fits catalog if input is path to the file
    if isinstance(s19a_fdfc_mask, str):
        cat = Table.read(cat)
        
    # Read the healpix mask if input is path to the file
    if isinstance(fdfc_mask, str):
        fdfc_mask = read_healpix_mask(fdfc_mask)
    
    # Find the matched objects
    nside, hp_indices = healpy.get_nside(fdfc_mask), np.where(fdfc_mask)[0]
    phi, theta = np.radians(cat[ra]), np.radians(90. - cat[dec])
    hp_masked = healpy.ang2pix(nside, theta, phi, nest=True)
    select = np.in1d(hp_masked, hp_indices)
    
    if verbose:
        print("# Find {:d} objects inside the FDFC region".format(select.sum()))
    
    if select.sum() < 1:
        return None
    return cat[select]

In [ ]:
sweep_list = list_all_sweep_catalogs(sweep_dir)

s19a_fdfc = read_healpix_mask(s19a_fdfc_mask)

matches = [filter_hsc_fdfc_mask(sweep, s19a_fdfc) for sweep in sweep_list]

# Find 50 SWEEP files
NSIDE = 1024
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
Ordering converted to NEST
# Find 48692 objects inside the FDFC region
# Find 23804 objects inside the FDFC region
# Find 3461144 objects inside the FDFC region
# Find 1486488 objects inside the FDFC region
# Find 0 objects inside the FDFC region
# Find 0 objects inside the FDFC region
# Find 0 objects inside the FDFC region
# Find 134507 objects inside the FDFC region
# Find 300026 objects inside the FDFC region
